In [1]:
# Leer Dataset

import pandas as pd

train = pd.read_csv("../files/input/train_data.csv.zip", index_col = False, compression = "zip")
test = pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")

In [2]:
# Año de referencia
reference_year = 2021

# Crear columna Age en ambos sets
train['Age'] = reference_year - train['Year']
test['Age'] = reference_year - test['Year']

In [3]:
# Remueva la columna "Year" y "Car_Name"
train.drop(columns=["Year","Car_Name"], inplace=True)
test.drop(columns=["Year","Car_Name"], inplace=True)

In [4]:
test.head()

,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
1,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
2,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
3,6.75,8.12,18796,Petrol,Dealer,Manual,0,6
4,6.50,8.61,33429,Diesel,Dealer,Manual,0,6


In [5]:
x_train = train.drop(columns=['Present_Price'])
y_train = train['Present_Price']

x_test = test.drop(columns=['Present_Price'])
y_test = test['Present_Price']

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

# Identificar columnas categóricas y numéricas
categoricas = ['Fuel_Type', 'Selling_type', 'Transmission']
numericas = ['Selling_Price', 'Driven_kms','Owner','Age']

# Preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categoricas),
        ('scaler', MinMaxScaler(), numericas),
    ],
)

#Selección de las k mejores características
selectkbest = SelectKBest(score_func=f_regression, k=10)

LinR= LinearRegression()

#Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', selectkbest),  
    ('classifier', LinR)  
])

In [7]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

CVkf = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'feature_selection__k': [5, 10, 'all'],
    'classifier__fit_intercept': [True, False]
}

model=GridSearchCV(
    pipeline,
    param_grid,
    cv=CVkf,
    scoring= "neg_mean_absolute_error",
    n_jobs=-1,
    refit=True
    )

# Entrenar el modelo
model.fit(x_train, y_train)

# Predicciones en el conjunto de prueba
y_pred = model.predict(x_test)


# Calcular el Error Medio Absoluto (MAE) en el conjunto de prueba
mae_test = mean_absolute_error(y_test, y_pred)

# Resultados
print(f"Mejores parámetros: {model.best_params_}")
print(f"Mejor MAE (validación cruzada): {-model.best_score_:.4f}")
print(f"MAE en el conjunto de prueba: {mae_test:.4f}")


Mejores parámetros: {'classifier__fit_intercept': False, 'feature_selection__k': 'all'}
Mejor MAE (validación cruzada): 1.8353
MAE en el conjunto de prueba: 2.4736


In [8]:
import gzip
import pickle
import os

# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
models_dir = '../files/models'
os.makedirs(models_dir, exist_ok=True)
with gzip.open("../files/models/model.pkl.gz", "wb") as file:
    pickle.dump(model, file)


In [9]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,median_absolute_error

# Realizar las predicciones sobre los conjuntos de entrenamiento y prueba
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Métricas train_data
r2_train = float(r2_score(y_train, y_train_pred))
mse_train = float(mean_squared_error(y_train, y_train_pred))
mad_train = float(median_absolute_error(y_train, y_train_pred))

# Métricas test_data
r2_test = float(r2_score(y_test, y_test_pred))
mse_test = float(mean_squared_error(y_test, y_test_pred))
mad_test = float(median_absolute_error(y_test, y_test_pred))

# Mostrar los resultados
print("Métricas para el conjunto de entrenamiento:")
print(f"R2: {r2_train}")
print(f"Error cuadrático medio: {mse_train}")
print(f"Error absoluto medio: {mad_train}")

print("\nMétricas para el conjunto de prueba:")
print(f"R2: {r2_test}")
print(f"Error cuadrático medio: {mse_test}")
print(f"Error absoluto medio: {mad_test}")

Métricas para el conjunto de entrenamiento:
R2: 0.8916962358587399
Error cuadrático medio: 5.8746462805982045
Error absoluto medio: 1.0929123440195554

Métricas para el conjunto de prueba:
R2: 0.7325716754123304
Error cuadrático medio: 32.56667275386628
Error absoluto medio: 1.503354060320583


In [10]:
import json
import os

metrics = [
    {
        'type': 'metrics',
        'dataset': 'train',
        'r2': r2_train,
        'mse': mse_train,
        'mad': mad_train,
    },
    {
        'type': 'metrics',
        'dataset': 'test',
        'r2': r2_test,
        'mse': mse_test,
        'mad': mad_test,
    },
]

metrics_path = "../files/output/metrics.json"
os.makedirs("../files/output", exist_ok=True)
with open(metrics_path, 'w') as f:
    for metric in metrics:
        f.write(json.dumps(metric, ensure_ascii=False))
        f.write('\n')
